In [ ]:
from neo4j import GraphDatabase
from datetime import datetime

In [20]:
# Connect to DB
URI = 'neo4j://localhost:7687/'
AUTH = ('neo4j', '123neo4j')

driver = GraphDatabase.driver(URI, auth=AUTH)

driver.verify_connectivity()

# Remove all data
driver.execute_query('MATCH (a) -[r] -> () DELETE a, r')
driver.execute_query('MATCH (a) DELETE a')

driver.execute_query(
    '''
Call dbms.listConfig() YIELD name, value
WHERE name='dbms.directories.neo4j_home'
RETURN value
    '''
)

# Import data from csv
# driver.execute_query(
#     '''
#     LOAD CSV WITH HEADERS FROM 'file:///items.csv' AS row
#     MERGE (p:Item {id: row.id, name: row.name, price: row.name});
#     '''
# )

# driver.execute_query(
#     '''
#     LOAD CSV WITH HEADERS FROM 'file:///customers.csv' AS row
#     MERGE (p:Customer {id: row.id, name: row.name});
#     '''
# )

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x10f9a3d40>, keys=['value'])

# Товари (items)